In [ ]:
!pip install kaggle


In [ ]:
from google.colab import files
files.upload()  # Upload the kaggle.json file here


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vignesh999","key":"da09f7ffb63a8de32a6b20046f2915a6"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Download the dataset to a specific folder in Google Drive
!kaggle datasets download -d sriramr/fruits-fresh-and-rotten-for-classification -p /content/drive/MyDrive/RFD


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset URL: https://www.kaggle.com/datasets/sriramr/fruits-fresh-and-rotten-for-classification
License(s): unknown
100% 3.58G/3.58G [01:05<00:00, 66.3MB/s]
100% 3.58G/3.58G [01:05<00:00, 58.4MB/s]


In [ ]:
!unzip /content/drive/MyDrive/RFD/fruits-fresh-and-rotten-for-classification.zip -d /content/drive/MyDrive/RFD/


In [ ]:
!pip install tensorflow opencv-python


In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Path to your dataset
dataset_path = '/content/drive/MyDrive/RFD/dataset/dataset/'

# Image size
IMG_SIZE = 128

# Function to load and preprocess images from all subfolders
def load_images_from_subfolders(base_folder):
    images = []
    labels = []
    class_labels = {'freshapples': 0, 'freshbanana': 1, 'freshoranges': 2,
                    'rottenapples': 3, 'rottenbanana': 4, 'rottenoranges': 5}

    for folder_name in os.listdir(base_folder):
        folder_path = os.path.join(base_folder, folder_name)
        if os.path.isdir(folder_path):
            label = class_labels[folder_name]
            for filename in os.listdir(folder_path):
                img = cv2.imread(os.path.join(folder_path, filename))
                if img is not None:
                    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                    images.append(img)
                    labels.append(label)

    return np.array(images), np.array(labels)

# Load training data
train_images, train_labels = load_images_from_subfolders(os.path.join(dataset_path, 'train'))

# Load testing data
test_images, test_labels = load_images_from_subfolders(os.path.join(dataset_path, 'test'))

# Data Augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(train_images, train_labels, batch_size=32)
test_generator = test_datagen.flow(test_images, test_labels, batch_size=32)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')  # Change to 6 for multiclass classification
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [ ]:
history = model.fit(train_generator, epochs=1, validation_data=test_generator)
model.save('fruit_ripening_model.h5')



341/341 ━━━━━━━━━━━━━━━━━━━━ 69s 186ms/step - accuracy: 0.5549 - loss: 1.1706 - val_accuracy: 0.8781 - val_loss: 0.3401


In [34]:
!apt-get install xvfb

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common xvfb
0 upgraded, 9 newly installed, 0 to remove and 45 not upgraded.
Need to get 7,813 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfontenc1 amd64 1:1.1.4-1build3 [14.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxfont2 amd64 1:2.0.5-1build1 [94.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxkbfile1 amd64 1:1.1.0-1build3 [71.8 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 x11-xkb-utils amd64 7.7+5build4 [172 kB]
Get:5 http://archiv

In [1]:
import tkinter as tk
from tkinter import filedialog
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image

# Load the trained model
model = load_model('fruit_ripening_model.h5')

# Define image size
IMG_SIZE = 128

def process_image(image_path):
    # Open the image and convert to RGB
    image = Image.open(image_path).convert('RGB')

    # Convert the image to a numpy array and resize it
    image_np = np.array(image)
    image_resized = cv2.resize(image_np, (IMG_SIZE, IMG_SIZE))
    image_resized = np.expand_dims(image_resized, axis=0) / 255.0

    # Make a prediction
    prediction = model.predict(image_resized)
    class_label = np.argmax(prediction[0])  # Assuming 6 classes

    return class_label

def upload_image():
    # Create a Tkinter root window
    root = tk.Tk()
    root.withdraw()  # Hide the root window

    # Open a file dialog to select an image file
    file_path = filedialog.askopenfilename(
        title="Select an Image",
        filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")]
    )

    if file_path:
        # Process the image and print the prediction
        class_label = process_image(file_path)
        print(f'Predicted class: {class_label}')
    else:
        print("No file selected.")

if __name__ == "__main__":
    upload_image()


TclError: no display name and no $DISPLAY environment variable

In [2]:
import tkinter as tk
from tkinter import filedialog
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image
import os

# Load the trained model
model = load_model('fruit_ripening_model.h5')

# Define image size
IMG_SIZE = 128

def process_image(image_path):
    # Open the image and convert to RGB
    image = Image.open(image_path).convert('RGB')

    # Convert the image to a numpy array and resize it
    image_np = np.array(image)
    image_resized = cv2.resize(image_np, (IMG_SIZE, IMG_SIZE))
    image_resized = np.expand_dims(image_resized, axis=0) / 255.0

    # Make a prediction
    prediction = model.predict(image_resized)
    class_label = np.argmax(prediction[0])  # Assuming 6 classes

    return class_label

def upload_image():
    # Create a virtual display
    if os.environ.get("DISPLAY", "") == "":
        print("no display found. Using :0.0")
        os.environ.__setitem__("DISPLAY", ":0.0")

    # Create a Tkinter root window
    root = tk.Tk()
    root.withdraw()  # Hide the root window

    # Open a file dialog to select an image file
    file_path = filedialog.askopenfilename(
        title="Select an Image",
        filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")]
    )

    if file_path:
        # Process the image and print the prediction
        class_label = process_image(file_path)
        print(f'Predicted class: {class_label}')
    else:
        print("No file selected.")

if __name__ == "__main__":
    upload_image()

no display found. Using :0.0


TclError: couldn't connect to display ":0.0"

In [3]:
import tkinter as tk
from tkinter import filedialog
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image
import os

# Load the trained model
model = load_model('fruit_ripening_model.h5')

# Define image size
IMG_SIZE = 128

def process_image(image_path):
    # Open the image and convert to RGB
    image = Image.open(image_path).convert('RGB')

    # Convert the image to a numpy array and resize it
    image_np = np.array(image)
    image_resized = cv2.resize(image_np, (IMG_SIZE, IMG_SIZE))
    image_resized = np.expand_dims(image_resized, axis=0) / 255.0

    # Make a prediction
    prediction = model.predict(image_resized)
    class_label = np.argmax(prediction[0])  # Assuming 6 classes

    return class_label

def upload_image():
    # Use try-except block to handle the exception
    try:
        # Create a Tkinter root window
        root = tk.Tk()
        root.withdraw()  # Hide the root window

        # Open a file dialog to select an image file
        file_path = filedialog.askopenfilename(
            title="Select an Image",
            filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")]
        )

        if file_path:
            # Process the image and print the prediction
            class_label = process_image(file_path)
            print(f'Predicted class: {class_label}')
        else:
            print("No file selected.")
    except tk.TclError:
        print("Tkinter GUI failed. You might be running in a headless environment.")
        print("Consider using a different method for image selection.")

if __name__ == "__main__":
    upload_image()

Tkinter GUI failed. You might be running in a headless environment.
Consider using a different method for image selection.


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image

# Load the trained model
model = load_model('fruit_ripening_model.h5')
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) # Recompile the model

# Define image size
IMG_SIZE = 128

def process_image(image_path):
    # Open the image and convert to RGB
    image = Image.open(image_path).convert('RGB')

    # Convert the image to a numpy array and resize it
    image_np = np.array(image)
    image_resized = cv2.resize(image_np, (IMG_SIZE, IMG_SIZE))
    image_resized = np.expand_dims(image_resized, axis=0) / 255.0

    # Make a prediction
    prediction = model.predict(image_resized)
    class_label = np.argmax(prediction[0])  # Assuming 6 classes

    return class_label

def upload_image():
    # Get the image path from the user
    file_path = input("Enter the path to your image: ")

    if file_path:
        # Process the image and print the prediction
        class_label = process_image(file_path)
        print(f'Predicted class: {class_label}')
    else:
        print("No file path entered.")

if __name__ == "__main__":
    upload_image()

In [32]:
!pip install pyvirtualdisplay

In [21]:
!pip install opencv-python

In [25]:
from google.colab import files
uploaded = files.upload()

import cv2
import numpy as np
from PIL import Image
from io import BytesIO

for filename in uploaded.keys():
    # Load and convert the image to RGB
    image = Image.open(BytesIO(uploaded[filename])).convert('RGB')
    image_np = np.array(image)

    # Resize and preprocess the image
    image_resized = cv2.resize(image_np, (IMG_SIZE, IMG_SIZE))
    image_resized = np.expand_dims(image_resized, axis=0) / 255.0

    # Make a prediction
    prediction = model.predict(image_resized)
    print("Prediction:", prediction)


Saving Keep it simple.png to Keep it simple (1).png
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 673ms/step
Prediction: [[1.4065854e-12 4.7610506e-08 1.7168195e-12 1.7954470e-05 9.9998081e-01
  1.1500632e-06]]
